In [4]:
import boto3
client = boto3.client('batch')
def submit_aws_job(*args, local=False, **kwargs):
    if not local:
        return client.submit_job(*args, **kwargs)

In [5]:
def submit_match(a, b, pool, side, timeout=600 , local=False):
    return submit_aws_job(
        jobDefinition='run-match',
        jobName=pool,
        jobQueue='matches',
        containerOverrides={
            'command': ["python", "simulate.py"],
            'environment': [
                {'name': 'PLAYER_A', 'value': a % 0},
                {'name': 'PLAYER_B', 'value': b % 1},
                {'name': 'POOL','value': pool},
                {'name': 'SIDE', 'value': str(side)},
            ]
        },
        timeout={'attemptDurationSeconds': timeout},
        local=local,
    )

In [7]:
def sumbit_symmetric_match(a, b, pool, timeout=600, local=False):
    submit_match(a, b, pool, side=0, timeout=timeout, local=local)
    #submit_match(b, a, pool, side=1, timeout=timeout, local=local)

In [8]:
sumbit_symmetric_match("GreedyPlayer(%s)", "GreedyPlayer(%s)", "tests-3")